In [13]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import glob
import pickle
import warnings
from scipy import misc
import scipy.ndimage
from skimage import feature
from PIL import Image
import os
import cv2
import math
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Lambda, Input, Conv2DTranspose, Reshape, UpSampling2D, BatchNormalization, Activation, Concatenate
from keras.models import model_from_json, Model
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping
from keras import backend as K
import keras
from scipy.stats import norm
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model

from keras.optimizers import Adam

In [14]:
def get_train_hollow_data(procent, model_number):
    train_hollow_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/hollow/"
    
    train_data_hollow = []
    
    for filename in os.listdir(train_hollow_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_hollow.append(img)
            
            
    np_train_data_hollow = np.array(train_data_hollow)
    
    train_data = np_train_data_hollow.reshape(np_train_data_hollow.shape[0], 128, 128, 1)
    
    return train_data

In [15]:
def get_train_metal_data(procent, model_number):
    train_metal_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/metal/"
    
    train_data_metal = []
    
    for filename in os.listdir(train_metal_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_metal.append(img)
            
            
    np_train_data_metal = np.array(train_data_metal)
    
    train_data = np_train_data_metal.reshape(np_train_data_metal.shape[0], 128, 128, 1)
    
    return train_data

In [16]:
def get_train_perfect_data(procent, model_number):
    train_perfect_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/perfect/"
    
    train_data_perfect = []
    
    for filename in os.listdir(train_perfect_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_perfect.append(img)
            
            
    np_train_data_perfect = np.array(train_data_perfect)
    
    train_data = np_train_data_perfect.reshape(np_train_data_perfect.shape[0], 128, 128, 1)
    
    return train_data

In [17]:
def create_generator(input_layer, condition_layer, img_shape):
    
    # Input to Dense
    merged_input = Concatenate()([input_layer, condition_layer])
    generator = Dense(256, activation='relu')(merged_input)
    generator = Dense(16*4*4, activation='relu')(generator)
    generator = Reshape((4, 4, 16)) (generator)
    generator = LeakyReLU(alpha=0.2) (generator)
    
    # Conv Layer 0
    generator = keras.layers.Conv2DTranspose(filters=16, kernel_size=[3,3], padding="same",
                                             strides=[2,2])(generator)
    #generator = BatchNormalization(momentum=0.8) (generator)
    generator = keras.layers.LeakyReLU(alpha=0.2) (generator)
    
    # Conv Layer 1
    generator = keras.layers.Conv2DTranspose(filters=16, kernel_size=[3,3], padding="same",
                                             strides=[2,2])(generator)
    generator = BatchNormalization(momentum=0.8) (generator)
    generator = keras.layers.LeakyReLU(alpha=0.2) (generator)
    
    
    # Conv Layer 2
    generator = keras.layers.Conv2DTranspose(filters=16, kernel_size=[3,3], padding="same", strides=[2,2])(generator)
    generator = BatchNormalization(momentum=0.8, name="batch_trans_conv2") (generator)
    generator = keras.layers.LeakyReLU(alpha=0.2) (generator)

    
    # Conv Layer 3
    generator = keras.layers.Conv2DTranspose(filters=16, kernel_size=[3,3], padding="same", strides=[2,2])(generator)
    generator = BatchNormalization(momentum=0.8,name="batch_trans_conv3") (generator)
    generator = keras.layers.LeakyReLU(alpha=0.2)(generator)
    
    
    # Conv Layer 4
    generator = keras.layers.Conv2DTranspose(filters=16, kernel_size=[3,3], padding="same", strides=[2,2]) (generator)
    generator = BatchNormalization(momentum=0.8,name="batch_trans_conv4") (generator)
    generator = keras.layers.LeakyReLU(alpha=0.2) (generator)
    
    
    # Output Conv
    generator = keras.layers.Conv2DTranspose(filters=1, kernel_size=[3,3], padding="same", strides=[1,1])(generator)
    out = Activation("sigmoid")(generator)
    
    model = Model(inputs=[input_layer, condition_layer], outputs=out)
    #model.summary()
  
    return model, out

In [18]:
def create_discriminator(input_layer, condition_layer, img_shape):
    # Small float added to variance to avoid dividing by zero when batch normalization
    EPSILON = 0.00005
    
    
    # Conv Layer 1
    discriminator = Conv2D(filters = 16, kernel_size=[3,3],strides=[2,2], padding="same",input_shape = img_shape)(input_layer)
    discriminator = BatchNormalization(momentum=0.8)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    
    # Conv Layer 2
    discriminator = Conv2D(filters=16, kernel_size=[3,3], strides=[2,2], padding="same")(discriminator)
    discriminator = BatchNormalization(momentum=0.8)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)
    
    
    # Conv Layer 3
    discriminator = Conv2D(filters=16, kernel_size=[3,3], strides=[2,2], padding="same")(discriminator)
    discriminator = BatchNormalization(momentum=0.8)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)
    
    
    # Conv Layer 4
    discriminator = Conv2D(filters=16, kernel_size=[3,3], strides=[2,2], padding="same")(discriminator)
    discriminator = BatchNormalization(momentum=0.8,epsilon = EPSILON)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    # Conv Layer 5
    discriminator = Conv2D(filters=16, kernel_size=[5,5], strides=[1,1], padding="same")(discriminator)
    discriminator = BatchNormalization(momentum=0.8,epsilon = EPSILON)(discriminator)
    discriminator = LeakyReLU(alpha=0.2)(discriminator)

    # Output layer
    discriminator = Flatten()(discriminator)
    merged_layer = Concatenate()([discriminator, condition_layer])
    out = Dense(1, activation='sigmoid')(merged_layer)

    model = Model(inputs=[input_layer, condition_layer], outputs=out)
    #model.summary()
    
    return model,out

In [19]:
def show_samples(sample_images, name, epoch):
    figure, axes = plt.subplots(1, len(sample_images), figsize = (128, 128))
    figure.set_size_inches(15,15)
    for index, axis in enumerate(axes):
        #print(index, axis)
        image_array = sample_images[index]
        image_array = np.array(image_array).reshape(128, 128)
        axis.imshow(image_array)
    plt.savefig(name+"_"+str(epoch)+".png", bbox_inches='tight', pad_inches=0)
    plt.tight_layout()
    plt.show()
    plt.close()
    
def summarize_epoch(d_losses, g_losses):
    fig, ax = plt.subplots()
    plt.plot(d_losses, label='Discriminator', alpha=0.6)
    plt.plot(g_losses, label='Generator', alpha=0.6)
    plt.title("Losses")
    plt.legend()
    plt.show()
    plt.close()
    
def get_batches(data, batch_size):
    batches = []
    for i in range(int(data.shape[0]//batch_size)):
        batch = data[i * batch_size:(i + 1) * batch_size]
        batches.append(batch)
    return batches

In [20]:
def one_hot_encode(y, latent_dim):
    z = np.zeros((len(y), latent_dim))
    idx = np.arange(len(y))
    z[idx, y] = 1
    return z

def generate_noise(n_samples, noise_size):
    X = np.random.normal(0, 1, size=(n_samples, noise_size))
    return X

def generate_random_labels(n, batch_size, latent_dim):
    y = np.random.choice(batch_size, n)
    y = one_hot_encode(y, latent_dim)
    return y

In [21]:
def run_model(input_images, procent, model, potato_type, number_of_images, batch_size, epochs, start_to_save):
    # Hyperparameters
    image_size = 128
    image_shape = (128, 128, 1)
    latent_dim = 50
    lr_discriminator = 0.004
    lr_generator = 0.004
    batch_size = input_images.shape[0]
    weight_init_stddev = 0.02
    epochs = epochs
    eps = 0.00005
    beta1 = 0.5
    samples_to_show = 5
    data_path = "potato_dataset/data_" + str(procent) + "/model_" + str(model) + "/gan_3/" + potato_type + "/"
    
    warnings.filterwarnings("ignore")
    # GAN creation
    img_input = Input(shape=image_shape)
    disc_condition_input = Input(shape=(10,))

    #Build the Discriminator
    discriminator, disc_out = create_discriminator(img_input, disc_condition_input, image_shape)
    discriminator.compile(optimizer=Adam(lr=lr_discriminator, beta_1=beta1), loss='binary_crossentropy', metrics=['accuracy'])

    discriminator.trainable = False

    #Build the Generator
    noise_input = Input(shape=(latent_dim,))
    gen_condition_input = Input(shape=(10,))
    generator, gen_out = create_generator(noise_input, gen_condition_input, image_shape)

    #Build GAN
    gan_input = Input(shape=(latent_dim,))
    gan_process = generator([gan_input, gen_condition_input])
    gan_output = discriminator([gan_process, disc_condition_input])
    gan = Model(inputs=[gan_input, gen_condition_input, disc_condition_input], outputs=gan_output)
    #gan.summary()

    gan.compile(optimizer=Adam(lr=lr_generator, beta_1=beta1), loss='binary_crossentropy')
    
    

    

    input_images = np.asarray(input_images)
    np.random.shuffle(input_images)
    
    
    warnings.filterwarnings("ignore")
    exp_replay = []
    batch_count = len(input_images)
    image_number = 0
    for epoch in range(epochs):

        cum_d_loss = 0.
        cum_g_loss = 0.
        d_losses = []
        g_losses = []
        start_time = time.time()

        for batch_idx in range(1):
            # Get the next set of real images to be used in this iteration
            images = input_images[batch_idx*batch_size : (batch_idx+1)*batch_size]
            labels = np.zeros((batch_size,10))

            noise_data = generate_noise(batch_size, latent_dim)
            random_labels = generate_random_labels(batch_size, batch_size, latent_dim)
            # We use same labels for generated images as in the real training batch
            generated_images = generator.predict([noise_data, labels])

            # Train on soft targets (add noise to targets as well)
            noise_prop = 0.05 # Randomly flip 5% of targets

            # Prepare labels for real data
            true_labels = np.zeros((batch_size, 1)) + np.random.uniform(low=0.0, high=0.1, size=(batch_size, 1))
            flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
            true_labels[flipped_idx] = 1 - true_labels[flipped_idx]

            # Train discriminator on real data
            discriminator.trainable = True
            d_loss_true = discriminator.train_on_batch([images, labels], true_labels)

            # Prepare labels for generated data
            gene_labels = np.ones((batch_size, 1)) - np.random.uniform(low=0.0, high=0.1, size=(batch_size, 1))
            flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
            gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]

            # Train discriminator on generated data
            d_loss_gene = discriminator.train_on_batch([generated_images, labels], gene_labels)

            # Store a random point for experience replay
            r_idx = np.random.randint(batch_size)
            exp_replay.append([generated_images[r_idx], labels[r_idx], gene_labels[r_idx]])

            #If we have enough points, do experience replay
            if len(exp_replay) == batch_size:
                generated_images = np.array([p[0] for p in exp_replay])
                labels = np.array([p[1] for p in exp_replay])
                gene_labels = np.array([p[2] for p in exp_replay])
                expprep_loss_gene = discriminator.train_on_batch([generated_images, labels], gene_labels)
                exp_replay = []
                break

            d_loss = 0.5 * np.add(d_loss_true, d_loss_gene)
            cum_d_loss += d_loss[0]
            d_losses.append(d_loss[0])

            # Train generator
            noise_data = generate_noise(batch_size, latent_dim)
            random_labels = np.zeros((batch_size,10))
            discriminator.trainable = False
            g_loss = gan.train_on_batch([noise_data, random_labels, random_labels], np.zeros((batch_size, 1)))
            cum_g_loss += g_loss
            g_losses.append(g_loss)

        # Uncomment if you want visulazation of Images
        '''
        if epoch % 200 == 0:
            summarize_epoch(d_losses, g_losses)
            #print(batch_count)
            print('\tEpoch: {}, Generator Loss: {}, Discriminator Loss: {}, Time: {}'.format(epoch+1, cum_g_loss/batch_count,
                                                                             cum_d_loss/batch_count, time.time()-start_time))
            samples = generator.predict([noise_data[:samples_to_show], np.ones((samples_to_show,10))])
            sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
            show_samples(sample_images, OUTPUT_DIR + "samples", 0)
        '''
        if(epoch >= start_to_save) and (epoch % 200 == 0):
            noise_data = generate_noise(number_of_images, latent_dim)
            labels = np.zeros((number_of_images,10))
            # We use same labels for generated images as in the real training batch
            generated_images = generator.predict([noise_data, labels])
            for noise_img in generated_images:
                plt.imsave(data_path + potato_type +"_gan_" + str(image_number) + ".jpg", noise_img.reshape(128, 128))
                image_number += 1
    K.clear_session()

In [ ]:
for p in range(10, 100, 10):
    start1 = time.time()
    for j in range(1, 11):
        start2 = time.time()
        hollow_data = get_train_hollow_data(p, j)
        run_model(hollow_data, p, j, "hollow", 10 + int(p / 10), hollow_data[0], 1600 + (p * 10) , 600 + (p * 10))
        end2 = time.time()
        print("Hollow Model {} with {} procent of the data took {} second".format(j, p, end2 - start2))
        
        start2 = time.time()
        metal_data = get_train_metal_data(p, j)
        run_model(metal_data, p, j, "metal", 10 + int(p / 10), metal_data[0], 1600 + (p * 10) , 600 + (p * 10))
        end2 = time.time()
        print("metal Model {} with {} procent of the data took {} second".format(j, p, end2 - start2))
        
        start2 = time.time()
        perfect_data = get_train_perfect_data(p, j)
        run_model(perfect_data, p, j, "perfect", 10 + int(p / 10), perfect_data[0], 1600 + (p * 10) , 600 + (p * 10))
        end2 = time.time()
        print("perfect Model {} with {} procent of the data took {} second".format(j, p, end2 - start2))

    end1 = time.time()
    print("VAE for {} procent of data took {} second".format(p, end1 - start1))